## 257. Introduction
- Go ahead and install PostgreSQL to get ready
- For windows users
    - Select your operating system
    - use 'Download the installer' option to download installer
- When you install, make sure that you enter valid username and password which you remember it (superuser: postgres, password: postgres)
    - Once the installation finishes, goto your commandline and you can use the following command
    ``` bash
    psql -U userName
    ```
    - ```psql``` is the PostgreSQL client or commandline tool, and then after above command, it'll prompt you for the password to emnter the command line using which we can create database, create users, create tables and perform CRUD operations

## 258. Setup the database
- You'll setup the ```employeedb``` Postgres database and ```employee``` table  inside it
- You'll then create a user 'test' with password 'password' and allow him to access our database and our table, so that we can use that user to connect to the database from within our node application
``` sql
    -- creates database
    CREATE DATABASE employeedb;
    -- uses database employeedb
    \c employeedb;
    -- creates table
    CREATE TABLE employee (id SERIAL PRIMARY KEY, name VARCHAR(30), sal REAL);
    -- creates user(role)
    CREATE USER test WITH ENCRYPTED PASSWORD 'password';
    -- grant database level privileges to user 'test'
    GRANT ALL PRIVILEGES ON DATABASE employeedb TO test;
    -- grant table permission to user 'test'
    GRANT ALL PRIVILEGES ON TABLE employee TO test;
    -- grant permission of sequence created internally to user 'test'
    -- postgres internally creates a sequnce named 'employee_id_seq' , to which user 'test' should have
    -- otherwise it'll show error that 'test' user does not have access to the sequence rendering us
    -- unable to increment the value of 'id' column when you use the user 'test'
    GRANT ALL PRIVILEGES ON SEQUENCE employee_id_seq TO test;
    -- shows all records from employee table
    SELECT * FROM employee;

    -- DROP DATABASE employeedb
    -- DROP USER test
```
- currently, it does not show any records, but we'll add records from our python code

## 259. Install psycopg2
-